In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import torch
import torch.nn as nn
import torch.optim as optim
import os

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.models import resnet18,googlenet
from sklearn.model_selection import train_test_split
from PIL import Image

In [21]:
def multiple_dir_files_to_df(dirs, label):
    df = pd.DataFrame()
    for idx,dir in enumerate(dirs):
        files = os.listdir(dir)
        files = [dir + '/' + file for file in files]
        df = pd.concat([df,pd.DataFrame({'file': files, 'label': label[idx]})])
    return df

common_path="Database/"

labels=["neutral","happy","sad","fear","angry","surprise","disgust","Happily_Surprised",
"Happily_Disgusted","Sadly_Fearful","Sadly_Angry","Sadly_Surprised","Sadly_Disgusted",
"Fearfully_Angry","Fearfully_Surprised","Fearfully_Disgusted","Angrily_Surprised",
"Angrily_Disgusted","Disgustedly_Surprised","Appalled","Hatred","Awed"]

database=[common_path+label for label in labels]

df = multiple_dir_files_to_df(database,np.linspace(0,len(labels)-1,len(labels)))
X,y=df["file"].tolist(),df["label"].tolist()

In [22]:
# Définir un Dataset personnalisé
class CustomDataset(Dataset):
    def __init__(self, file_names, labels, transform=None):
        self.file_names = file_names
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_names)

    def __getitem__(self, idx):
        img_path = f'{self.file_names[idx]}' 
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label


In [23]:
# Define the image transformations
transform = transforms.Compose([
    transforms.CenterCrop((500, 500)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [24]:
class GoogleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.googlenet = googlenet(pretrained=True)
        self.googlenet.fc = nn.Linear(512, 7)
        self.googlenet.inception4d=nn.Identity()
        self.googlenet.inception4e=nn.Identity()
        self.googlenet.inception5a=nn.Identity()
        self.googlenet.inception5b=nn.Identity()
        self.googlenet.Linear=nn.Identity()

    def forward(self, x):
        return self.googlenet(x)

model= GoogleNet()
print(model)

c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


GoogleNet(
  (googlenet): GoogLeNet(
    (conv1): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): BasicConv2d(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (inception3a): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn

In [25]:
# Définir le modèle ResNet
class ResNetModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnet = resnet18(pretrained=True)
        self.resnet.fc = nn.Linear(256, 7)
        self.resnet.layer4= nn.Identity()
        self.resnet.layer1[-1].bn2 = nn.Sequential(
            self.resnet.layer1[-1].bn2,
            nn.Dropout2d(p=0.15)
        )

        self.resnet.layer2[-1].bn2 = nn.Sequential(
            self.resnet.layer2[-1].bn2,
            nn.Dropout2d(p=0.15)
        )

        self.resnet.layer3[-1].bn2 = nn.Sequential(
            self.resnet.layer3[-1].bn2,
            nn.Dropout2d(p=0.15)
        )
        self.resnet.fc=nn.Identity()
        
    def forward(self, x):
        return self.resnet(x)

model= ResNetModel()
print(model)

c:\Users\Daito\miniconda3\envs\ContinualLearning\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNetModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_

In [26]:
# Créer les jeux de données et les chargeurs de données
t_dataset = CustomDataset(X, y, transform)
t_loader = DataLoader(t_dataset, batch_size=10, shuffle=False)

In [31]:
#model = ResNetModel()  
#model.load_state_dict(torch.load(pretrained_model, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')),strict=False)
print(os.getcwd())

c:\Users\Daito\OneDrive\Bureau\CapProjetRecherche\code\ML_model_implementation\Features_extraction


In [36]:
#Features extraction
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pretrained_model ="./resnet_18_pretrained_7_emo_19.pth"
model= torch.load(pretrained_model, map_location=device)
model.eval()
df=pd.DataFrame()

for i, (x, y) in enumerate(t_loader):
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        output = model(x)
        df=pd.concat([df,pd.DataFrame(output.cpu().numpy())])
        print(df.shape)
        


In [ ]:
df.to_csv("features_resnet.csv",index=False)

In [ ]:
print(df.head())

        0         1         2         3         4         5         6    \
0  0.058971  0.626277  0.107093  0.670639  0.685376  0.672881  0.541003   
1  0.293999  0.680076  0.070964  0.704674  0.800976  0.759914  0.789037   
2  0.026243  0.493064  0.071549  0.598101  0.668996  0.798525  0.383728   
3  0.264287  0.605679  0.043540  0.597127  0.845713  0.770380  0.757597   
4  0.049339  0.610061  0.145537  0.661091  0.692952  0.739890  0.485597   

        7         8         9    ...       246       247       248       249  \
0  0.021652  0.078406  0.713660  ...  0.555141  0.029386  0.000524  0.013478   
1  0.083484  0.022216  0.857469  ...  0.636875  0.287480  0.177322  0.007050   
2  0.031116  0.151202  0.682578  ...  0.351739  0.033807  0.009727  0.029941   
3  0.022491  0.014760  0.863484  ...  0.605871  0.238986  0.094417  0.003430   
4  0.096431  0.157701  0.787261  ...  0.477616  0.060900  0.035032  0.053462   

        250       251       252       253       254       255  
0  0